In [56]:
!pip install Faker
!pip install fake-useragent


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import random
import time
import requests
from bs4 import BeautifulSoup
from faker import Faker
import csv
from fake_useragent import UserAgent

base_url = "https://www.amazon.com/dp/"


# headers = {
#           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
#           'Accept-Language': 'da, en-gb, en',
#           'Accept-Encoding': 'gzip, deflate, br',
#           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
#           'Referer': 'https://www.google.com/'
#         }


def get_soup_retry(url):
    # fake = Faker()
    # uag_random = fake.user_agent()
    ua = UserAgent(browsers=['edge', 'chrome', 'firefox', 'safari'], platforms='pc')
    uag_random = ua.random

    header = {
        'User-Agent': uag_random,
        'Accept-Language': 'en-US,en;q=0.9'
    }
    isCaptcha = True
    while isCaptcha:
        #time.sleep(random.uniform(0.5, 1))
        page = requests.get(url, headers=header)
        if not page.status_code == 200:
            print(f"page status code error: {page.status_code}")
        soup = BeautifulSoup(page.content, 'html.parser')
        if 'captcha' in str(soup):
            uag_random = ua.random #fake.user_agent()
            print(f'\rBot has been detected... retrying ... use new identity: {uag_random} ', end='', flush=True)
            continue
        else:
            print('Bot bypassed')
            return soup, page.status_code


def get_description(soup):
    div_class_description = "a-expander-content a-expander-partial-collapse-content"
    outer_div = soup.find('div', {'data-a-expander-name': 'book_description_expander'})
    
    if outer_div:
        inner_div = outer_div.find('div', class_=div_class_description)
        if inner_div:
            inner_text = inner_div.get_text(strip=True, separator=' ')
            return inner_text
        else:
            return None
    else:
        return None

def get_new_values(url):
    soup, status_code = get_soup_retry(url)
    
    description = get_description(soup)
    if description is None:
        print(f"Description not found for url: {url}")
        description = ""
    return {"description" : description, "code" : status_code}

def get_asin(row):
    asin = row[0]
    img_name = row[1]
    # fix "761183272","0761183272.jpg" -> "0761183272","0761183272.jpg"
    if len(asin) < (len(img_name) - len(".jpg")) and (asin[0] != '0' and img_name[0] == '0'):
        asin = "0" + asin
    return asin
    

def process_csv_no_header(input_csv, output_csv, start_row, end_row, rnd = 0):
    with open(input_csv, mode='r', encoding='utf-8', errors='ignore') as infile, open(output_csv, mode='w', encoding='utf-8', newline='') as outfile:
        # Read input as plain rows
        reader = csv.reader(infile, quotechar='"')
        writer = csv.writer(outfile, quotechar='"', quoting=csv.QUOTE_MINIMAL) # make sure to correctly escape characters

        reader_list = list(reader)
        random_list = []
        if rnd > 0:
            number_rows = len(reader_list)
            random_list = sorted([random.randint(0, number_rows - 1) for _ in range(rnd)])
            # print(random_list)

        for row_index, row in enumerate(reader_list):
            # Process rows within the range
            if (rnd > 0 and (row_index in random_list)) or (rnd == 0 and start_row <= row_index < end_row):
                # print(row)
                if len(row) >= 7:
                    # Check if there is an 8th and 9th field for description and status_code
                    if len(row) >= 9:
                        description = row[7].strip()
                        # Replace description only if empty
                        if not description:
                            asin = get_asin(row)  # Assuming ASIN is in the first column
                            new_values = get_new_values(base_url + asin)
                            row[7] = new_values["description"]
                            row[8] = new_values["code"]
                    else:
                        # Add a new 8th and 9th field if not present
                        asin = get_asin(row)  # Assuming ASIN is in the first column
                        new_values = get_new_values(base_url + asin)
                        row.append(new_values["description"])
                        row.append(new_values["code"])
            # Write the row to the output CSV
            writer.writerow(row)


In [33]:
input_csv_path = "book32-listing.csv"
output_csv_path = "book32-listing_0_100.csv"

# Specify range of rows to process (e.g., rows 0–10)
process_csv_no_header(input_csv_path, output_csv_path, start_row=0, end_row=100, rnd=0)
# if rnd value is set and > 0 the values of start_row and end_row are not considered, we draw rnd rows from the csv file and scrape

Bot has been detected... retrying ... use new identity: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Trailer/93.3.3516.28 Bot bypassed
Bot has been detected... retrying ... use new identity: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0 Bot bypassed
Bot has been detected... retrying ... use new identity: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0 Bot bypassed
Description not found for url: https://www.amazon.com/dp/B00O80WC6I
Bot has been detected... retrying ... use new identity: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0 Chrome/122.0.0.0 Safari/537.36 Bot bypassed
Bot has been detected... retrying ... use new identity: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0 page status code error

In [1]:
import random
import time
import requests
from bs4 import BeautifulSoup
from faker import Faker
import csv
from fake_useragent import UserAgent
from concurrent.futures import ThreadPoolExecutor
from threading import Lock
from tqdm import tqdm # progress bar

base_url = "https://www.amazon.com/dp/"
lock = Lock()  # Ensures thread-safe writing to the CSV file
description_counter = 0  # Global counter for descriptions found

def get_soup_retry(url):
    ua = UserAgent(browsers=['edge', 'chrome', 'firefox', 'safari'], platforms='pc')
    uag_random = ua.random
    header = {
        'User-Agent': uag_random,
        'Accept-Language': 'en-US,en;q=0.9'
    }
    isCaptcha = True
    while isCaptcha:
        page = requests.get(url, headers=header)
        # if not page.status_code == 200:
        #     print(f"Page status code error: {page.status_code}")
        soup = BeautifulSoup(page.content, 'html.parser')
        if 'captcha' in str(soup):
            uag_random = ua.random
            # print(f'\rBot detected... retrying with new identity: {uag_random} ', end='', flush=True)
            continue
        else:
            # print('Bot bypassed')
            return soup, page.status_code

def get_description(soup):
    div_class_description = "a-expander-content a-expander-partial-collapse-content"
    outer_div = soup.find('div', {'data-a-expander-name': 'book_description_expander'})
    if outer_div:
        inner_div = outer_div.find('div', class_=div_class_description)
        if inner_div:
            return inner_div.get_text(strip=True, separator=' ')
    return None

def get_new_values(url):
    soup, status_code = get_soup_retry(url)
    description = get_description(soup) or ""
    return {"description": description, "code": status_code}

def get_asin(row):
    asin = row[0]
    img_name = row[1]
    if len(asin) < (len(img_name) - len(".jpg")) and (asin[0] != '0' and img_name[0] == '0'):
        asin = "0" + asin
    return asin

def process_row(row, writer, progress_bar):
    global description_counter
    if len(row) >= 7:
        if len(row) >= 9:
            if not row[7].strip():  # Replace empty description
                asin = get_asin(row)
                new_values = get_new_values(base_url + asin)
                row[7] = new_values["description"]
                row[8] = new_values["code"]
                if new_values["description"]:
                    with lock:
                        description_counter += 1
        else:
            asin = get_asin(row)
            new_values = get_new_values(base_url + asin)
            row.append(new_values["description"])
            row.append(new_values["code"])
            if new_values["description"]:
                with lock:
                    description_counter += 1
        # Write row with a lock to avoid race conditions
        with lock:
            writer.writerow(row)
    progress_bar.update(1)

def process_csv_parallel(input_csv, output_csv, start_row, end_row, rnd=0, max_workers=5):
    global description_counter
    description_counter = 0 # reset
    with open(input_csv, mode='r', encoding='utf-8', errors='ignore') as infile, \
         open(output_csv, mode='w', encoding='utf-8', newline='') as outfile:
        reader = csv.reader(infile, quotechar='"')
        writer = csv.writer(outfile, quotechar='"', quoting=csv.QUOTE_MINIMAL)

        reader_list = list(reader)
        random_list = []
        if rnd > 0:
            number_rows = len(reader_list)
            random_list = sorted([random.randint(0, number_rows - 1) for _ in range(rnd)])

        total_rows = (end_row - start_row) if rnd == 0 else len(random_list)
        with tqdm(total=total_rows, desc="Processing Rows") as progress_bar:
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                futures = []
                for row_index, row in enumerate(reader_list):
                    if (rnd > 0 and row_index in random_list) or (rnd == 0 and start_row <= row_index < end_row):
                        futures.append(executor.submit(process_row, row, writer, progress_bar))
    
                for future in futures:
                    future.result()  # Ensure all tasks complete before exiting
    print(f"\nDescriptions found: {description_counter} / {end_row - start_row}")

In [4]:
new_input_csv_path = "book30-listing-train.csv"
new_output_csv_path = "book30-listing-train_0_100.csv"

# Specify range of rows to process (e.g., rows 0–10)
process_csv_parallel(new_input_csv_path, new_output_csv_path, start_row=0, end_row=700, rnd=0, max_workers=4)
# if rnd value is set and > 0 the values of start_row and end_row are not considered, we draw rnd rows from the csv file and scrape

Processing Rows: 100%|██████████| 700/700 [04:29<00:00,  2.59it/s]


Descriptions found: 4 / 700


In [ ]:
#concatenate csv files

In [12]:
import pandas as pd
import glob

# Define the pattern for your CSV files
file_pattern = "data/book_descriptions_test_*.csv"

# Sort files by the numeric range in their names
csv_files = sorted(glob.glob(file_pattern), key=lambda x: int(x.split('_')[-1].split('.')[0].split('-')[0]))
# print(csv_files)

# Initialize an empty list to hold the dataframes
dataframes = []


for i, file in enumerate(csv_files):
    if i == 0:
        # Read the first file including the header, handle quotes and line breaks
        df = pd.read_csv(file, quotechar='"', skip_blank_lines=True, engine="python")
    else:
        # Read subsequent files, skipping the first line (header)
        df = pd.read_csv(file, skiprows=1, quotechar='"', skip_blank_lines=True, engine="python")
    
    # Ensure numeric columns like "200" remain integers and drop empty trailing columns
    df = df.convert_dtypes()  # Infer correct column types
    df = df.dropna(how="all")  # Drop rows where all elements are NaN

    dataframes.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged DataFrame to a single CSV file
merged_df.to_csv("data/book_description_test_complete.csv", index=False)


In [ ]:
# clean text

In [11]:
import csv
import re

# Function to clean control characters from a string
def remove_control_characters(text):
    # Remove all control characters (non-printable characters)
    # return re.sub(r'[^\x20-\x7E\n\r\t]', '', text)  # Keep spaces, printable characters, newlines, and tabs
    return re.sub(r'[\x00-\x1F\x7F]', '', text)  # Remove ASCII control characters

# Function to clean CSV file
def clean_csv(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        for row in reader:
            # Clean each cell in the row
            cleaned_row = [remove_control_characters(cell) for cell in row]
            writer.writerow(cleaned_row)
    
    print(f"Cleaned CSV has been saved to {output_file}")

# Example usage
input_file = 'book30-listing-test.csv'  # Replace with your input CSV file
output_file = 'cleaned_output_1.csv'  # Replace with your desired output CSV file
clean_csv(input_file, output_file)


Cleaned CSV has been saved to cleaned_output_1.csv


In [ ]:
# merge csv files

In [10]:
import csv

# File paths
file1 = 'book30-listing-train.csv'  # First file path
file2 = 'data/book_descriptions_train.csv'  # Second file path
output_file = '../data/full_data/book_descriptions_train_full.csv'  # Output file path
unmatched_ids_file = 'unmatched_ids_test2.txt'  # File to save unmatched IDs

# Column names for both files
file1_headers = ['id', 'img_name', 'img_url', 'title', 'author', 'category_id', 'category']
file2_headers = ['id', 'description', 'status_code']

# Read the second file into a dictionary for quick lookup
second_file_data = {}
with open(file2, 'r', encoding='utf-8') as f2:  # Use utf-8 encoding
    reader = csv.reader(f2)
    for row in reader:
        row_data = dict(zip(file2_headers, row))
        second_file_data[row_data['id']] = row_data

# Open the first file and merge with the second file data
unmatched_ids = []
with open(file1, 'r', encoding='utf-8') as f1, open(output_file, 'w', newline='', encoding='utf-8') as out_file:
    reader = csv.reader(f1)
    writer = csv.writer(out_file)
    
    # Write the header for the output file
    output_headers = file1_headers + ['description', 'status_code']
    writer.writerow(output_headers)
    
    for row in reader:
        row_data = dict(zip(file1_headers, row))
        id_ = row_data['id']
        img_name = row_data['img_name']
        if (len(img_name) - len(".jpg")) > len(id_) and img_name[0] == "0":
            id_ = img_name[:-4]
            row_data.update({'id': id_})
        if id_ in second_file_data:
            # Merge rows
            row_data.update(second_file_data[id_])
        else:
            # Record unmatched ID
            unmatched_ids.append(id_)
            row_data.update({'description': '', 'status_code': ''})
        
        # Write the merged row
        writer.writerow([row_data[col] for col in output_headers])

# Write unmatched IDs to a text file
# with open(unmatched_ids_file, 'w', encoding='utf-8') as unmatched_file:
#     for id_ in unmatched_ids:
#         unmatched_file.write(f"{id_}\n")

print(f"Merged data written to {output_file}")
print(f"Unmatched IDs written to {unmatched_ids_file}")


Merged data written to ../data/full_data/book_descriptions_train_full.csv
Unmatched IDs written to unmatched_ids_test2.txt


In [ ]:
# get some statistics

In [16]:
import pandas as pd

def gather_statistics(input_file, output_file):
    # Load the CSV file into a Pandas DataFrame
    df = pd.read_csv(input_file, encoding='utf-8')
    
    # Total number of entries
    total_entries = len(df)
    
    # Entries with non-empty description
    entries_with_description = len(df[df['description'].notna() & (df['description'] != '')])
    
    # Number of entries per category
    category_counts = df['category'].value_counts()
    
    # Number of entries with non-empty description per category
    category_with_description_counts = df[df['description'].notna() & (df['description'] != '')]['category'].value_counts()
    
    # Find the category with the least entries with non-empty description
    least_entries_category = category_with_description_counts.idxmin()
    least_entries_count = category_with_description_counts.min()
    
    # Prepare the results in a string format for writing to file
    output = []
    output.append(f"Total entries: {total_entries}\n")
    output.append(f"Entries with non-empty description: {entries_with_description}\n")
    
    output.append("Entries per category:\n")
    output.append(category_counts.to_string())
    
    output.append("\nEntries with non-empty description per category:\n")
    output.append(category_with_description_counts.to_string())
    
    output.append(f"\nCategory with least entries with non-empty description: {least_entries_category}")
    output.append(f"Number of entries: {least_entries_count}")
    
    # Write the statistics to the output file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("\n".join(output))
    
    print(f"Statistics have been written to {output_file}")

# Example usage
input_file = '../data/full_data/book_descriptions_test_balanced.csv'  # Replace with your merged CSV file
output_file = '../data/stats/book_descriptions_test_balanced.txt'  # Replace with your desired output text file
gather_statistics(input_file, output_file)


Statistics have been written to ../data/stats/book_descriptions_test_balanced.txt


In [ ]:
# Balance dataset

In [15]:
import pandas as pd

def clean_and_balance_dataset(input_file, output_file):
    # Load the CSV file
    df = pd.read_csv(input_file, encoding='utf-8')
    
    # Step 1: Remove entries with category_id = 3
    df = df[df['category_id'] != 3]
    
    # Step 2: Decrement category_id by 1 for all category_id >= 4
    df['category_id'] = df['category_id'].apply(lambda x: x - 1 if x >= 4 else x)
    
    # Step 3: Filter out rows without descriptions (keep only rows with non-empty descriptions)
    df['description'] = df['description'].fillna('')  # Replace NaN descriptions with empty strings
    df = df[df['description'].apply(lambda x: x.strip() != '')]  # Keep only rows with non-empty descriptions
    
    # Step 4: Count non-empty descriptions for each category
    category_description_counts = df['category_id'].value_counts()
    
    # Step 5: Find the smallest number of non-empty descriptions across all categories
    min_descriptions = category_description_counts.min()
    print(f"Category with the least non-empty descriptions has {min_descriptions} entries.")
    
    # Step 6: Balance the dataset by randomly sampling entries from each category
    balanced_dfs = []
    for category_id, group in df.groupby('category_id'):
        # Randomly sample from the rows to match `min_descriptions`
        sampled_group = group.sample(n=min_descriptions, random_state=42)
        balanced_dfs.append(sampled_group)
    
    # Concatenate all balanced dataframes
    balanced_df = pd.concat(balanced_dfs)
    
    # Step 7: Save the cleaned and balanced dataset to a new CSV file
    balanced_df.to_csv(output_file, index=False, encoding='utf-8')
    
    # Output statistics
    print(f"Cleaned and balanced dataset saved to {output_file}")
    print(f"Final dataset contains {len(balanced_df)} entries.")

# Example usage
input_file = '../data/full_data/book_descriptions_test_full.csv'  # Replace with your input CSV file
output_file = '../data/full_data/book_descriptions_test_balanced.csv'  # Replace with your desired output CSV file
clean_and_balance_dataset(input_file, output_file)


Category with the least non-empty descriptions has 164 entries.
Cleaned and balanced dataset saved to ../data/full_data/book_descriptions_test_balanced.csv
Final dataset contains 4756 entries.


In [ ]:
# create train and test dataset

In [18]:
import pandas as pd

def prepare_training_dataset(input_file, output_file):
    # Load the dataset
    df = pd.read_csv(input_file, encoding='utf-8')
    
    # Step 1: Select only the relevant columns
    df = df[['img_name', 'description', 'category_id']]
    
    # Step 2: Limit the description to 500 characters
    # df['description'] = df['description'].apply(lambda x: x[:500] if isinstance(x, str) else x)
    
    # Step 3: Shuffle the dataset randomly
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Step 4: Save the simplified dataset to a new CSV file
    df.to_csv(output_file, index=False, encoding='utf-8')
    
    print(f"Training dataset saved to {output_file}")
    print(f"Total entries in the training dataset: {len(df)}")

# Example usage
input_file = '../data/full_data/book_descriptions_train_balanced.csv'  # Replace with your balanced dataset file
output_file = '../data/book_descriptions_train.csv'  # Replace with your desired output file
prepare_training_dataset(input_file, output_file)


Training dataset saved to ../data/book_descriptions_train.csv
Total entries in the training dataset: 41470


In [ ]:
# check for missing img files

In [ ]:
import os
import pandas as pd

def find_missing_images(folder_path, csv_file_path):
    # Step 1: Get all file names in the folder
    folder_files = set(os.listdir(folder_path))
    
    # Step 2: Load the CSV file and extract the img_name column
    df = pd.read_csv(csv_file_path, encoding='utf-8')
    csv_images = set(df['img_name'])
    
    # Step 3: Find img_names in the CSV that are not in the folder
    missing_images = csv_images - folder_files
    
    # Step 4: Print the missing img_names
    if missing_images:
        print("Missing images:")
        for img_name in missing_images:
            print(img_name)
    else:
        print("All images in the CSV are present in the folder.")

# Example usage
folder_path = '../data/images'  # Replace with the path to your folder
csv_file_path = '../data/book_descriptions_test.csv'   # Replace with the path to your CSV file
find_missing_images(folder_path, csv_file_path)


In [ ]:
# remove image files that we are not using

In [ ]:
import os
import pandas as pd

def remove_unmatched_images(folder_path, csv_file1, csv_file2):
    # Step 1: Load both CSV files
    df1 = pd.read_csv(csv_file1, encoding='utf-8')
    df2 = pd.read_csv(csv_file2, encoding='utf-8')
    
    # Step 2: Get the set of all valid image names from both CSVs
    valid_img_names = set(pd.concat([df1['img_name'], df2['img_name']]).dropna())
    
    # Step 3: Iterate through files in the folder and remove unmatched ones
    for file_name in os.listdir(folder_path):
        if file_name not in valid_img_names:
            file_path = os.path.join(folder_path, file_name)
            os.remove(file_path)
            print(f"Removed: {file_name}")
    
    print("Cleanup complete. All unmatched files have been removed.")

# Example usage
folder_path = '../data/images'    # Replace with your folder path
csv_file1 = '../data/book_descriptions_test.csv'          # Replace with the path to your first CSV file
csv_file2 = '../data/book_descriptions_train.csv'          # Replace with the path to your second CSV file
remove_unmatched_images(folder_path, csv_file1, csv_file2)
